In [4]:
import numpy as np
import matplotlib.pyplot as plt
import freud
import signac
import gsd
import hoomd
import gsd.pygsd
import gsd.hoomd
import scipy
from scipy import signal
import time
import os

In [5]:
ptb7_proj = signac.get_project("ptb7-project")
itic_proj = signac.get_project("ptb7-project")

In [16]:
def atom_type_pos(frame, atom_types): 
    positions = []
    for idx, type_id in enumerate(frame.particles.typeid):
        if frame.particles.types[type_id] in atom_types:
            positions.append(frame.particles.position[idx])
    return positions


def rdf_from_gsd(gsd_file, atom_types='all', r_max=5, r_min=0.5, nbins=50, start=95):
    f = gsd.pygsd.GSDFile(open(gsd_file, "rb"))
    #trajectory = gsd.hoomd.open(gsd_file, 'rb')
    trajectory = gsd.hoomd.HOOMDTrajectory(f)
    
    if r_max is None:
        r_max = max(trajectory[-1].configuration.box[:3]) * 0.45
    freud_rdf = freud.density.RDF(bins=nbins, r_max=r_max, r_min=r_min)
    for frame in trajectory[start:]:
        if atom_types == 'all':
            freud_rdf.compute(frame, reset=False)
        else:
            query_points = atom_type_pos(frame, atom_types)
            box = frame.configuration.box
            freud_rdf.compute(system=(box, query_points), reset=False)
            x = freud_rdf.bin_centers
            y = freud_rdf.rdf
            filename = "{}/trajectory.gsd".format(job.workspace())  
    np.savetxt(filename, np.transpose([x,y]), delimiter=',', header= "bin_centers, rdf")
    f.close()
    return freud_rdf

def msd_slope(polymer_length, temp, density, column = ["all", "sulfur", "centers"], step_size = 1):
    
    filename = "{}/msd.log".format(job.workspace())
    msd = np.genfromtxt(filename, names = True, delimiter = ",") 
    x = msd['timestep']
    y = msd[atom_type]
    
    num_data_points = len(x)
    start_index = num_data_points // 10
    x_slice = x[-start_index:0]
    y_slice = y[-start_index:0]    
    
    slope = (y_slice[-1] - y_slice[0]) / (x_slice[-1] - x_slice[0])
    return slope
    f.close()
    return


In [ ]:
def data_analysis(jobs=[], function=rdf()):
    for job in jobs:
        go do something
        if not job.isfile('rdf-data.txt'):
           function(job)
